In [ ]:
d = {'a':1}
d.get('a')

In [ ]:
import json
from textwrap import dedent as d
import dash
from dash.dependencies import Input, Output, State
import dash_core_components as dcc
import dash_html_components as html
import numpy as np

app = dash.Dash(__name__)
#app.css.append_css({'external_url': 'https://codepen.io/chriddyp/pen/dZVMbK.css'})

#styles = {'pre': {'border': 'thin lightgrey solid', 'overflowX': 'scroll'}}

dots_pos = [(1,1),(3,1),(5,1),(7,1)]
f=4
fs = 1000
N = 10000
trace_t = [i/fs for i in range(N)]

# Controller functions

def get_envelope_handles(shapes):
    dots = []
    for s in shapes:
        name = s['name'] or ''
        if name.startswith('envelope_'):
            dots.append((s['xanchor'], s['yanchor']))
    return dots

def get_num_envelope_handles(shapes):
    max_index = -1
    for s in shapes:
        name = s['name'] or ''
        if name.startswith('envelope_'):
            cur_index = int(name.strip('envelope_'))
            max_index = max(cur_index,max_index)
    return max_index + 1

def get_envelope_handle(shapes,index):
    for s in shapes:
        if s['name'] == 'envelope_' + str(index):
            return s
    return None


def add_envelope_handle(shapes, xmax):
    # Add a new envelope handle
    # Use same y value and halfway between the furthest x handle and a maximum
    num_handles = get_num_envelope_handles(shapes)
    shape_template = get_envelope_handle(shapes,num_handles-1).copy()
    shape_template.update(
        {   'name'   : 'envelope_'+str(num_handles),
            'xanchor':(shape_template['xanchor']+xmax)/2
        })
    shapes.append(shape_template)

def rm_envelope_handle(shapes, index=None):
    # Remove envelope_handle with supplied index (of with highest index)
    if index==None:
        index = get_num_envelope_handles(shapes)-1
        # Never implicitly delete the last remaining handle
        if index == 0:
            return
    
    for (i,s) in enumerate(shapes):
        if s['name'] == 'envelope_'+str(index):
            shapes.pop(i)

# Model functions

def calc_envelope(dots):
    sorted_dots = dots.copy()
    sorted_dots.sort(key=lambda x: x[0])
    env = np.interp(trace_t, [p[0] for p in sorted_dots],
                             [p[1] for p in sorted_dots])
    return env

ref_i = np.array([np.cos(2*np.pi*f*t) for t in trace_t])
ref_q = np.array([np.sin(2*np.pi*f*t) for t in trace_t])

env = calc_envelope(dots_pos)
trace_i = ref_i * env
trace_q = ref_q * env

app.layout = html.Div(className='row', children=[
    html.Button('Add Envelope Handle', id='btn-add-handle'),
    html.Button('Remove Envelope Handle', id='btn-rm-handle'),
    dcc.Graph(
        id='basic-interactions',
        className='six columns',
        figure={
            'data': [{
                'x': trace_t,
                'y': trace_i,
                'name': 'I',
                'mode': 'lines'
            }, {
                'x': trace_t,
                'y': trace_q,
                'name': 'q',
                'mode': 'lines'
            }],
            'layout': {
                'shapes': [{
                    'type': 'circle',
                    'name': 'envelope_'+str(i),
                    'x0': -5,
                    'x1': 5,
                    'xref': 'x',
                    'xanchor': x,
                    'xsizemode':'pixel',

                    'y0': -5,
                    'y1': 5,
                    'yref': 'y',
                    'yanchor': y,
                    'ysizemode':'pixel',
                    'fillcolor': 'rgb(165, 10, 235)',
                    'line': {
                        'width': 4,
                        'color': 'rgb(165, 10, 235)'
                    }
                } for (i,(x,y)) in enumerate(dots_pos)]
            }
        },
        config={
            'editable': True,
            'edits': {
                'shapePosition': True
            }
        }
    ),
    html.Div(
        className='six columns',
        children=[
            html.Div(
                [
                    dcc.Markdown(
                        d("""
                **Hello world**

            """)),
                    html.Pre(id='relayout-data', style=styles['pre']),
                ]
            )
        ]
    )
])

@app.callback(
    Output('basic-interactions', 'figure'),
    [Input('basic-interactions', 'relayoutData'),
     Input('btn-add-handle', 'n_clicks'),
     Input('btn-rm-handle', 'n_clicks')],
    [State('basic-interactions', 'figure')])
def display_selected_data(_,btn_add_handle,btn_rm_handle,fig):
    
    changed_id = [p['prop_id'] for p in dash.callback_context.triggered][0]
    if 'btn-add-handle' in changed_id:
        add_envelope_handle(fig['layout']['shapes'], max(trace_t))
    if 'btn-rm-handle' in changed_id:
        rm_envelope_handle(fig['layout']['shapes'])
    
    env = calc_envelope(get_envelope_handles(fig['layout']['shapes']))
    trace_i = ref_i * env
    trace_q = ref_q * env
    
    fig['data'][0]['x'] = trace_t
    fig['data'][0]['y'] = trace_i
    fig['data'][1]['x'] = trace_t
    fig['data'][1]['y'] = trace_q
    return fig
app.run_server(debug=False)

In [ ]:
max([1,2,3])

How should be interpolate between dots then?

Let's loop through each x value and do a straight line interpolation. Before first dot and after last dot are just constant amplitude.

In [ ]:
    
    #fig['layout']['shapes'].append({
    #                'type': 'vrect',
    #                'x0': 1,
    #                'x1': 2,
    #                'fillcolor': "LightSalmon",
    #                'opacity':0.5,
    #                'layer':"below",
    #                'line_width': 0
    #            })